### Imports

In [ ]:
import sys
sys.dont_write_bytecode = True


import torch
import numpy as np
import random
import os

def set_seeds(seed_value=42):
    """Sets seeds for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)

set_seeds(42) 

import json
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import warnings
import logging
from datetime import datetime
warnings.filterwarnings('ignore')

from model import get_model
from config import CFG
from dataset import get_dataset_class
from resnet.plot import plot_domainwise_accuracy
from transform import get_transforms
from runner import run_baseline, run_lodo

torch.manual_seed(CFG["system"]["seed"])
np.random.seed(CFG["system"]["seed"])

device = CFG["system"]["device"]
print(f"Device: {device}")
print(f"PyTorch: {torch.__version__}")

DS = "RMNIST"
MODEL_NAME = "resnet18"

Device: cuda
PyTorch: 2.5.1+cu121


### DataLoading

In [ ]:
train_transform, test_transform = get_transforms(img_size=224, augment=False, use_imagenet_norm=False)

DatasetClass = get_dataset_class(DS)

ld = DatasetClass(
    data_root=CFG["datasets"][DS]["root"],
    transform=train_transform,
    batch_size=CFG["train"]["batch_size"]
)

print("\nData loaders ready!")


Data loaders ready!


### Logging

In [ ]:
dataset_name = DS
base_dir = os.path.join(os.getcwd(), dataset_name)
subdirs = ["logs", "checkpoints", "plots"]

for sub in subdirs:
    os.makedirs(os.path.join(base_dir, sub), exist_ok=True)

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_file = os.path.join(base_dir, "logs", f"train_{timestamp}.log")

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger(f"{dataset_name}_logger")

logger.info(f"Initialized experiment directories for {dataset_name}")
logger.info(f"Logs: {os.path.join(base_dir, 'logs')}")
logger.info(f"Checkpoints: {os.path.join(base_dir, 'checkpoints')}")
logger.info(f"Plots: {os.path.join(base_dir, 'plots')}")

2025-10-10 23:52:42,401 | INFO | Initialized experiment directories for RMNIST
2025-10-10 23:52:42,401 | INFO | Logs: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\resnet\RMNIST\logs
2025-10-10 23:52:42,401 | INFO | Checkpoints: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\resnet\RMNIST\checkpoints
2025-10-10 23:52:42,401 | INFO | Plots: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\resnet\RMNIST\plots


### Setup

In [ ]:
domains = CFG["datasets"][DS]["domains"]
loaders = {d: {"train": ld.get_dataloader(d, train=True), "val": ld.get_dataloader(d, train=False)} for d in domains}
ckpt_root = os.path.join(base_dir, "checkpoints")
log_dir = os.path.join(base_dir, "logs")
plots_dir = os.path.join(base_dir, "plots")
os.makedirs(ckpt_root, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)
model_factory = lambda cfg, dataset_key: get_model(cfg,dataset=DS)
optimizer_fn = lambda model: optim.AdamW(model.parameters(), lr=CFG["train"]["lr"], weight_decay=CFG["train"].get("weight_decay", 0.01))
device = CFG["system"]["device"]
epochs = CFG["train"]["epochs"]

{
  "lodo_results": {
    "art_painting": 0.8341463414634146,
    "cartoon": 0.7974413646055437,
    "photo": 0.9580838323353293,
    "sketch": 0.6017811704834606
  },
  "timestamp": "20251004_020611"
}

### Leave One Domain Out

In [ ]:
lodo_results, lodo_mean, lodo_summary = run_lodo(
    model_fn=model_factory,
    CFG=CFG,
    logger=logger,
    dataset_key=DS,
    domains=domains,
    loaders=loaders,
    optimizer_fn=optimizer_fn,
    device=device,
    ckpt_root=ckpt_root,
    log_dir=log_dir,
    epochs=epochs
)

2025-10-10 20:45:08,259 | INFO | === LODO: Leaving out domain '0' ===



=== LODO: Leaving out domain '0' ===


Evaluating: 100%|██████████| 547/547 [00:28<00:00, 19.15it/s]
2025-10-10 20:50:21,641 | INFO | [0] Epoch 1/5 | Train - Loss: 0.0549, Cls: 0.0540, GRQO: 0.0009, Acc: 0.9841 | Val - Loss: 0.1301, Cls: 0.1301, GRQO: 0.0000, Acc: 0.9603
2025-10-10 20:50:21,692 | INFO | [0] New best val acc: 0.9603


[0] Epoch 1/5 | Train - Loss: 0.0549, Cls: 0.0540, GRQO: 0.0009, Acc: 0.9841 | Val - Loss: 0.1301, Cls: 0.1301, GRQO: 0.0000, Acc: 0.9603
[0] New best val acc: 0.9603


Evaluating: 100%|██████████| 547/547 [00:27<00:00, 19.69it/s]
2025-10-10 20:55:34,723 | INFO | [0] Epoch 2/5 | Train - Loss: 0.0193, Cls: 0.0192, GRQO: 0.0001, Acc: 0.9942 | Val - Loss: 0.0815, Cls: 0.0815, GRQO: 0.0000, Acc: 0.9775
2025-10-10 20:55:34,797 | INFO | [0] New best val acc: 0.9775


...

[30] Epoch 1/5 | Train - Loss: 0.0558, Cls: 0.0548, GRQO: 0.0009, Acc: 0.9841 | Val - Loss: 0.0293, Cls: 0.0293, GRQO: 0.0000, Acc: 0.9906
[30] New best val acc: 0.9906


Evaluating: 100%|██████████| 547/547 [00:27<00:00, 19.63it/s]
2025-10-10 21:48:00,338 | INFO | [30] Epoch 2/5 | Train - Loss: 0.0204, Cls: 0.0204, GRQO: 0.0001, Acc: 0.9936 | Val - Loss: 0.0477, Cls: 0.0477, GRQO: 0.0000, Acc: 0.9838


[30] Epoch 2/5 | Train - Loss: 0.0204, Cls: 0.0204, GRQO: 0.0001, Acc: 0.9936 | Val - Loss: 0.0477, Cls: 0.0477, GRQO: 0.0000, Acc: 0.9838


Evaluating: 100%|██████████| 547/547 [00:27<00:00, 19.65it/s]
2025-10-10 21:53:14,678 | INFO | [30] Epoch 3/5 | Train - Loss: 0.0145, Cls: 0.0145, GRQO: 0.0000, Acc: 0.9955 | Val - Loss: 0.0202, Cls: 0.0202, GRQO: 0.0000, Acc: 0.9942
2025-10-10 21:53:14,753 | INFO | [30] New best val acc: 0.9942


[30] Epoch 3/5 | Train - Loss: 0.0145, Cls: 0.0145, GRQO: 0.0000, Acc: 0.9955 | Val - Loss: 0.0202, Cls: 0.0202, GRQO: 0.0000, Acc: 0.9942
[30] New best val acc: 0.9942


### Baseline

In [ ]:
baseline_results, baseline_mean = run_baseline(
    model_name=MODEL_NAME,
    CFG=CFG,
    logger=logger,
    dataset_key=DS,
    domains=domains,
    loaders=loaders,
    optimizer_fn=optimizer_fn,
    device=device,
    epochs=CFG["train"]["epochs"]
)

2025-10-10 23:53:00,750 | INFO | Initializing ResNet baseline: resnet18
2025-10-10 23:53:00,901 | INFO | === Baseline LODO: Leaving out domain '0' ===


Initializing ResNet baseline: resnet18

=== Baseline LODO: Leaving out domain '0' ===


2025-10-10 23:57:23,704 | INFO | [0] Epoch 1/5 | Train - Loss: 0.0425, Acc: 0.9875 | Val Acc: 0.9643


[0] Epoch 1/5 | Train - Loss: 0.0425, Acc: 0.9875 | Val Acc: 0.9643


2025-10-11 00:02:48,819 | INFO | [0] Epoch 2/5 | Train - Loss: 0.0153, Acc: 0.9953 | Val Acc: 0.9725


...

2025-10-11 01:25:20,723 | INFO | [45] Epoch 5/5 | Train - Loss: 0.0064, Acc: 0.9979 | Val Acc: 0.9951
2025-10-11 01:25:20,723 | INFO | [45] Best Val Acc: 0.9951
2025-10-11 01:25:20,723 | INFO | ------------------------------------------------------------
2025-10-11 01:25:20,723 | INFO | Initializing ResNet baseline: resnet18
2025-10-11 01:25:20,823 | INFO | === Baseline LODO: Leaving out domain '60' ===


[45] Epoch 5/5 | Train - Loss: 0.0064, Acc: 0.9979 | Val Acc: 0.9951
[45] Best Val Acc: 0.9951
------------------------------------------------------------
Initializing ResNet baseline: resnet18

=== Baseline LODO: Leaving out domain '60' ===


2025-10-11 01:29:47,470 | INFO | [60] Epoch 1/5 | Train - Loss: 0.0434, Acc: 0.9874 | Val Acc: 0.9910


[60] Epoch 1/5 | Train - Loss: 0.0434, Acc: 0.9874 | Val Acc: 0.9910


KeyboardInterrupt: 